## Extract trial data

Takes output pickle (.pkl) files recorded with Titta and divides them into trials using messages sent during the experiment. Will output a folder with trial data in .csv-format (one file for each trial).

In [1]:
# import required libraries
import pickle
import numpy as np
import pandas as pd
import os
from pathlib import Path

In [2]:
def extract_trial_data(df_et_data, df_msg, msg_onset, msg_offset):
    ''' Extracts data from one trial associated with specific onset and offset messages.

    Args:
        df_et_data - Pandas dataframe with sample et-data (output from Titta)
        df_msg - Pandas dataframe containing messages
        msg_onset (str) - message sent at stimulus onset (e.g., onset_polarbear)
        msg_offset (str) - message sent at stimulus onset (e.g., offset_polarbear)


    Returns:
        df - dataframe with data from one trial

    '''

    # Find timestamps for data belonging to this stimulus
    start_idx = np.where(df_msg.msg == msg_onset)[0][0]
    stop_idx = np.where(df_msg.msg == msg_offset)[0][0]

    start_time_stamp = df_msg.system_time_stamp[start_idx]
    stop_time_stamp = df_msg.system_time_stamp[stop_idx]

    # print(start_idx, stop_idx, start_time_stamp, stop_time_stamp)
    #
    # Cut out samples belonging to this stimulus
    fix_idx_start = np.searchsorted(df_et_data.system_time_stamp,
                                    start_time_stamp)
    fix_idx_stop = np.searchsorted(df_et_data.system_time_stamp,
                                   stop_time_stamp)
    df_stim = df_et_data.iloc[fix_idx_start:fix_idx_stop].copy()

    return df_stim

In [3]:
# header / column names
header = ['device_time_stamp',
         'system_time_stamp',
         'left_gaze_point_on_display_area_x',
         'left_gaze_point_on_display_area_y',
         'left_gaze_point_in_user_coordinate_system_x',
         'left_gaze_point_in_user_coordinate_system_y',
         'left_gaze_point_in_user_coordinate_system_z',
         'left_gaze_origin_in_trackbox_coordinate_system_x',
         'left_gaze_origin_in_trackbox_coordinate_system_y',
         'left_gaze_origin_in_trackbox_coordinate_system_z',
         'left_gaze_origin_in_user_coordinate_system_x',
         'left_gaze_origin_in_user_coordinate_system_y',
         'left_gaze_origin_in_user_coordinate_system_z',
         'left_pupil_diameter',
         'left_pupil_validity',
         'left_gaze_origin_validity',
         'left_gaze_point_validity',
         'right_gaze_point_on_display_area_x',
         'right_gaze_point_on_display_area_y',
         'right_gaze_point_in_user_coordinate_system_x',
         'right_gaze_point_in_user_coordinate_system_y',
         'right_gaze_point_in_user_coordinate_system_z',
         'right_gaze_origin_in_trackbox_coordinate_system_x',
         'right_gaze_origin_in_trackbox_coordinate_system_y',
         'right_gaze_origin_in_trackbox_coordinate_system_z',
         'right_gaze_origin_in_user_coordinate_system_x',
         'right_gaze_origin_in_user_coordinate_system_y',
         'right_gaze_origin_in_user_coordinate_system_z',
         'right_pupil_diameter',
         'right_pupil_validity',
         'right_gaze_origin_validity',
         'right_gaze_point_validity']

# Read messages and et data all participants (one .pkl-file per participant)
files = Path.cwd().glob('*.pkl')

for f in files:

    pid = str(f).split(os.sep)[-1][:-4]

    fh = open(f, 'rb')
    gaze_data_container = pickle.load(fh)
    msg_container = pickle.load(fh)

    # Convert to pandas dataframes
    df = pd.DataFrame(gaze_data_container, columns=header)
    df_msg = pd.DataFrame(msg_container, columns=['system_time_stamp', 'msg'])

    # Read message for onset and offset
    # Assumption is that messages are on the form (must be unique)
    # 'onset_stimulusname' for the onset of a stimulus and
    # 'offset_stimulusname'for the offset of a stimulus
    onset = []
    offset = []
    for i, row in df_msg.iterrows():

        if 'onset' in row.msg:
            onset.append(row.msg)
        if 'offset' in row.msg:
            offset.append(row.msg)
    trial_msg = zip(onset, offset)

    # Create a folder to put the trials
    path = Path.cwd() / 'trials' / pid
    try:
        path.mkdir(parents=True, exist_ok=False)
    except FileExistsError:
        print("Folder is already there")
    else:
        print("Folder was created")

    # Extract relevant trial data and save in format required by I2MC
    for t in trial_msg:
        df_trial = extract_trial_data(df, df_msg, t[0], t[1])

        filename = t[0].split('_')[1] + '.tsv'
        df_trial.to_csv(str(path) + os.sep + filename, sep='\t')

        print('Trial ' + filename + " written to folder ", path)




Folder is already there
Trial im3.jpeg.tsv written to folder  c:\git\Titta\resources\trials\participant1
Trial im2.jpeg.tsv written to folder  c:\git\Titta\resources\trials\participant1
Trial im1.jpeg.tsv written to folder  c:\git\Titta\resources\trials\participant1
Folder is already there
Trial im3.jpeg.tsv written to folder  c:\git\Titta\resources\trials\participant2
Trial im2.jpeg.tsv written to folder  c:\git\Titta\resources\trials\participant2
Trial im1.jpeg.tsv written to folder  c:\git\Titta\resources\trials\participant2


## Detect fixations from extracted trial data

OBS! Before running the below cell, download and extract the I2MC algorithm (https://github.com/dcnieho/I2MC_Python) to the folder resources/I2MC. The name of the extracted folder must be 'I2MC_Python-master'.

Running the cell will prompt the user to download the I2MC algorithm (if not already done), and move trial data to the approriate folder in the I2MC folder structure.


### 1. Copy data to required I2MC folder

In [4]:
import shutil

I2MC_main_path = Path.cwd() / 'I2MC' / 'I2MC_Python-master' / 'example'

I2MC_main = I2MC_main_path / 'I2MC_example.py'
if not I2MC_main.is_file():
    print('It appears that I2MC is not available. please follow the\
 instructions in /resources/I2MC/get_I2MC.txt to download it.')
    raise FileNotFoundError

# First remove example data found when downloading I2MC
example_data_path = I2MC_main_path / 'example_data'
if example_data_path.is_dir():
    shutil.rmtree(example_data_path)
    print('Removed old data')


# Move trial data you extracted to the right folder I2MC
shutil.copytree(str(Path.cwd() / 'trials'), str(example_data_path))
print('Copied data to ' + str(example_data_path))

Removed old data
Copied data to c:\git\Titta\resources\I2MC\I2MC_Python-master\example\example_data


### 2. Run the I2MC algorithm

Open the jupyter notebook I2MC_example.ipynb located in resources\I2MC\I2MC_Python-master\example.
OBS! Before running the notebook, adjust the settings to match your experimental setup. Check under 'Options' in the beginning of the file. Also make sure Titta is use when importing data. Replace the line (in 'Start the algorithm' section)

data = imp.tobii_TX300(file_name, [opt['xres'], opt['yres']])

with

data = imp.Titta(file_name, [opt['xres'], opt['yres']])

